In [ ]:
!pip install nnunetv2

In [20]:
if not torch.cuda.is_available():
    print('You have to install CUDA to run this code')
else:
    print('True')

You have to install CUDA to run this code


In [1]:
import os
from batchgenerators.utilities.file_and_folder_operations import *

os.environ["nnUNet_raw"] = join('.')
os.environ["nnUNet_preprocessed"] = join('.')

# Put weights path(final directory) here. For example: /Users/mohannatd/Desktop/final
os.environ["nnUNet_results"] = input()

import torch
import shutil
import numpy as np
import torch.nn as nn
import nibabel as nib
import torch.nn.functional as F
from IPython.display import clear_output
from torchvision.models.vgg import vgg19

 /Users/mohannatd/Desktop/test/final


In [2]:
def percentage(part, whole):
  Percentage = 100 * part/whole
  return str(round(Percentage) + 1) + "%"

In [3]:
class VGG19(torch.nn.Module):
    def __init__(self, device='cuda'):
        super(VGG19, self).__init__()
        features = list(vgg19(pretrained=True).features)
        if device == "cuda":
            self.features = nn.ModuleList(features).cuda().eval()
        else:
            self.features = nn.ModuleList(features).eval()

    def forward(self, x):
        feature_maps = []
        for idx, layer in enumerate(self.features):
            x = layer(x)
            if idx == 3:
                feature_maps.append(x)
        return feature_maps


class Fusion:
    def __init__(self, input):
        """
        Class Fusion constructor

        Instance Variables:
            self.images: input images
            self.model: CNN model, default=vgg19
            self.device: either 'cuda' or 'cpu'
        """
        self.input_images = input
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = VGG19(self.device)

    def fuse(self):
        """
        A top level method which fuse self.images
        """
        # Convert all images to YCbCr format
        self.normalized_images = [-1 for img in self.input_images]
        self.YCbCr_images = [-1 for img in self.input_images]
        for idx, img in enumerate(self.input_images):
            if not self._is_gray(img):
                self.YCbCr_images[idx] = self._RGB_to_YCbCr(img)
                self.normalized_images[idx] = self.YCbCr_images[idx][:, :, 0]
            else:
                self.normalized_images[idx] = img / 255.
        # Transfer all images to PyTorch tensors
        self._tranfer_to_tensor()
        # Perform fuse strategy
        fused_img = self._fuse()[:, :, 0]
        # Reconstruct fused image given rgb input images
        for idx, img in enumerate(self.input_images):
            if not self._is_gray(img):
                self.YCbCr_images[idx][:, :, 0] = fused_img
                fused_img = self._YCbCr_to_RGB(self.YCbCr_images[idx])
                fused_img = np.clip(fused_img, 0, 1)

        return (fused_img * 255).astype(np.uint8)

    def _fuse(self):
        """
        Perform fusion algorithm
        """
        with torch.no_grad():

            imgs_sum_maps = [-1 for tensor_img in self.images_to_tensors]
            for idx, tensor_img in enumerate(self.images_to_tensors):
                imgs_sum_maps[idx] = []
                feature_maps = self.model(tensor_img)
                for feature_map in feature_maps:
                    sum_map = torch.sum(feature_map, dim=1, keepdim=True)
                    imgs_sum_maps[idx].append(sum_map)

            max_fusion = None
            for sum_maps in zip(*imgs_sum_maps):
                features = torch.cat(sum_maps, dim=1)
                weights = self._softmax(F.interpolate(features,
                                        size=self.images_to_tensors[0].shape[2:]))
                weights = F.interpolate(weights,
                                        size=self.images_to_tensors[0].shape[2:])
                current_fusion = torch.zeros(self.images_to_tensors[0].shape)
                for idx, tensor_img in enumerate(self.images_to_tensors):
                    current_fusion += tensor_img * weights[:,idx]
                if max_fusion is None:
                    max_fusion = current_fusion
                else:
                    max_fusion = torch.max(max_fusion, current_fusion)

            output = np.squeeze(max_fusion.cpu().numpy())
            if output.ndim == 3:
                output = np.transpose(output, (1, 2, 0))
            return output

    def _RGB_to_YCbCr(self, img_RGB):
        """
        A private method which converts an RGB image to YCrCb format
        """
        img_RGB = img_RGB.astype(np.float32) / 255.
        return cv2.cvtColor(img_RGB, cv2.COLOR_RGB2YCrCb)

    def _YCbCr_to_RGB(self, img_YCbCr):
        """
        A private method which converts a YCrCb image to RGB format
        """
        img_YCbCr = img_YCbCr.astype(np.float32)
        return cv2.cvtColor(img_YCbCr, cv2.COLOR_YCrCb2RGB)

    def _is_gray(self, img):
        """
        A private method which returns True if image is gray, otherwise False
        """
        if len(img.shape) < 3:
            return True
        if img.shape[2] == 1:
            return True
        b, g, r = img[:,:,0], img[:,:,1], img[:,:,2]
        if (b == g).all() and (b == r).all():
            return True
        return False

    def _softmax(self, tensor):
        """
        A private method which compute softmax ouput of a given tensor
        """
        tensor = torch.exp(tensor)
        tensor = tensor / tensor.sum(dim=1, keepdim=True)
        return tensor

    def _tranfer_to_tensor(self):
        """
        A private method to transfer all input images to PyTorch tensors
        """
        self.images_to_tensors = []
        for image in self.normalized_images:
            np_input = image.astype(np.float32)
            if np_input.ndim == 2:
                np_input = np.repeat(np_input[None, None], 3, axis=1)
            else:
                np_input = np.transpose(np_input, (2, 0, 1))[None]
            if self.device == "cuda":
                self.images_to_tensors.append(torch.from_numpy(np_input).cuda())
            else:
                self.images_to_tensors.append(torch.from_numpy(np_input))

In [5]:
def zero_learning_fusion(input_images):
  FU = Fusion(input_images)
  fusion_img = FU.fuse()
  return fusion_img

In [4]:
def normalizer(image):
    img_min = np.min(image)
    img_max = np.max(image)
    image = (image - img_min ) / ( img_max - img_min)
    image = image * 255
    return image

In [8]:
def fusion(raw_data_path):
    base_dir = raw_data_path

    fusion_modes = ['t1n-t1c', 't1c-t2f', 't1c-t2w']
    images = []
    done = []
    for i, mode in enumerate(fusion_modes):
        Final_fusion3D = ''
        images = [nib.load(join(base_dir, '{}.nii.gz'.format(mode.split('-')[0]))).get_fdata()]
        images.append(nib.load(join(base_dir, '{}.nii.gz'.format(mode.split('-')[1]))).get_fdata())
        for slice_num in range(155):
            if (slice_num%10) == 0:
              clear_output()
              print(percentage(155*i + slice_num + 1, 155*3))
              print('Wait until the process ends')
              for do in done:
                  print(do + ': Done')
            slices = [normalizer(image[:,:,slice_num]) for image in images]
            fusion = zero_learning_fusion(slices)
            if slice_num == 0:
                Final_fusion3D = fusion
            else:
                Final_fusion3D = np.dstack((Final_fusion3D ,fusion))
    
        output = nib.Nifti1Image(Final_fusion3D, np.eye(4))
        output.header.get_xyzt_units()
        output.to_filename(join(base_dir, '{}.nii.gz'.format(mode)))
        done.append(mode)
        if mode == 't1c-t2w':
            print(mode + ': Done')

In [11]:
'''Put raw input path with the following names here.
For example: /Users/mohannatd/Desktop/raw_input

/Users/mohannatd/Desktop/raw_input :
|__ t1n.nii.gz
|__ t1c.nii.gz
|__ t2f.nii.gz
|__ t2w.nii.gz

'''
input_path = input()

fusion(input_path)

 /Users/mohannatd/Desktop/inp


In [ ]:
imagestr = join('.', 'input')

try:
    os.mkdir(imagestr)
except:
    pass

shutil.copy(join(input_path, "t1n-t1c.nii.gz"), join(imagestr, 'Brain_0000.nii.gz'))
shutil.copy(join(input_path, "t2f.nii.gz"), join(imagestr, 'Brain_0001.nii.gz'))
shutil.copy(join(input_path, "t1c-t2w.nii.gz"), join(imagestr, 'Brain_0002.nii.gz'))
shutil.copy(join(input_path, "t1c-t2f.nii.gz"), join(imagestr, 'Brain_0003.nii.gz'))

output_path = join(input_path, 'output')

!nnUNetv2_predict -i $imagestr -o $output_path -d 13 -c 3d_fullres -f all -tr nnUNetTrainerDiceCELoss_noSmooth
